In [56]:
import requests
import json
import pandas as pd
import time
from IPython.display import clear_output, display
import sys
import random
import numpy as np
import os
import psycopg2
# Strategy
# get list of players
# from list of players, get list of games
# from list of games, get game stats
from utils import apikey, payload, rate, regions
import utils
from sqlalchemy import create_engine

dbname = 'league'
engine = create_engine(f"postgresql+psycopg2://{utils.pgUsername}:{utils.pgPassword}@/{dbname}?host={utils.pgHost}")
#conn = psycopg2.connect(dbname=dbname,user=utils.pgUsername,password=utils.pgPassword,host=utils.pgHost)
#cursor = conn.cursor()

In [48]:
conn.commit()


In [50]:
conn.close()


In [126]:
def writeGameInit(pd_table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """    
    print(keys)
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:        
        query = f"INSERT INTO {pd_table.name} (gameid, gamecreation, queueid, platformid, region, tier) VALUES "
        vals = list()
        for data in data_iter:
            gameid = data[keys.index('gameid')]
            gamecreation = data[keys.index('timestamp')]
            queueid = data[keys.index('queue')]
            platformid = data[keys.index('platformid')]
            region = data[keys.index('region')]
            tier = data[keys.index('tier')]
            vals.append(f"({gameid}, {gamecreation}, {queueid}, '{platformid}', '{region}', '{tier}')")
        query += ", ".join(vals) + " ON CONFLICT ON CONSTRAINT games_gameid_key DO NOTHING;
        display(query)
        cur.execute(query)
        

In [127]:
df.to_sql('games_temp', engine,if_exists='append',method=writeGameInit)

['index', 'gameid', 'queue', 'timestamp', 'platformid', 'tier', 'region']


"INSERT INTO games_temp (gameid, gamecreation, queueid, platformid, region, tier) VALUES (3640364171, 1604096511411, 420, 'NA1', 'na1', 'GOLD'), (3640361136, 1604095205900, 420, 'NA1', 'na1', 'GOLD'), (3639806211, 1604053436032, 420, 'NA1', 'na1', 'GOLD'), (3638051956, 1603955769905, 420, 'NA1', 'na1', 'GOLD'), (3637954785, 1603952603352, 420, 'NA1', 'na1', 'GOLD'), (3637868432, 1603950046861, 420, 'NA1', 'na1', 'GOLD'), (3634845717, 1603781303647, 420, 'NA1', 'na1', 'GOLD'), (3634836839, 1603778527489, 420, 'NA1', 'na1', 'GOLD'), (3634791343, 1603775650292, 420, 'NA1', 'na1', 'GOLD'), (3633743604, 1603706136029, 420, 'NA1', 'na1', 'GOLD'), (3633770080, 1603703610938, 420, 'NA1', 'na1', 'GOLD'), (3632197251, 1603617214497, 420, 'NA1', 'na1', 'GOLD'), (3632241076, 1603614257080, 420, 'NA1', 'na1', 'GOLD'), (3632097606, 1603611868203, 420, 'NA1', 'na1', 'GOLD'), (3632100987, 1603609082146, 420, 'NA1', 'na1', 'GOLD'), (3632016040, 1603607229339, 420, 'NA1', 'na1', 'GOLD'), (3631958766, 16

In [9]:
result = None
with engine.connect() as connection:
    query = f"SELECT * from summoners where region='{region}' and summonerid='DcqJLORFwEYSUuNlfxkSmNNVGX4DDAs3PQ-ffY8IixP3j33z'"
    result = connection.execute(query)
records = result.fetchall()
df = pd.DataFrame(records,columns=result.keys())

In [43]:
summonerId = summonerids[0]
query = f"SELECT * FROM summoners WHERE summonerid='{summonerId}' and region='{region}'"
cursor.execute(query)
result = cursor.fetchone()
cols = [d[0] for d in cursor.description]
print(cols)
print(result)

['summonerid', 'summonername', 'tier', 'rank', 'region', 'accountid', 'id', 'last_accessed', 'puuid']
('U7d5Iw3xkzChNqCzKgT7q48ucGl6zDZvox0grlfiYjz5E1A', 'Just Stibi', 'MASTER', 'I', 'na1', 'tIs5yJ0qgQAGM6VTjKVxXSps9Y_QqNIA5uFWNkBhnNrlRRspJGqSLeJj', 1446459, datetime.datetime(2020, 10, 26, 18, 13, 6, 967488), 'RibKMg3wdgbuKxlpJLVBxotp_C_iItU8cvXEqSpvH8tCxBjsyKJRgoRFsx20Dt2f88b6yy5Q7P54aA')


In [58]:
records = None
with engine.connect() as connection:
    result = engine.execute('SELECT * from summoners where last_accessed is not null')
    records = result.fetchone()

In [59]:
records

('O8NK7bY6rTtglhoY1KQYCK1ro3FPMJPQK1iO6p-xaGoKyQ', '엉덩국 갱승제로', 'DIAMOND', 'I', 'kr', 'i3TMPWxUi5nP8k-Aj_jm1yVDI97VxvV_OXwgS-L0XEwg', 123397, datetime.datetime(2020, 11, 2, 19, 4, 38, 834799), 'a2NeaY7jTugAzkyROF1VOrVNPvK6WuUiveTEncrYsisEkiNwnDaacvqWJlwRyETcQWZ1zZ_oWW0AkA')

In [61]:
records['last_accessed'].timestamp()

1604372678.834799